# **Homework Assignment #5 Solution**

Assigned: March 24, 2021

Due: April 12, 2021



---

This assignment consists of questions that require a short answer and one Python programming task. You can enter your answers and your code directly in a Colaboratory notebook and upload the **shareable** link for the your notebook as your homework submission.


---

#1.

 (12 points) In most real world scenarios, data contain outliers. When using a support vector machine, outliers can be dealt with using a soft margin, specified in a slightly different optimization problem shown in Equation 7.38 in the text and called a soft-margin SVM.

Intuitively, where does a data point lie relative to the margin when $\zeta_i = 0$? Is this data point classified correctly?

Intuitively, where does a data point lie relative to the margin when $0 < \zeta_i \leq 1$? Is this data point classified correctly?

Intuitively, where does a data point lie relative to the margin when $\zeta_i > 1$? Is this data point classified correctly?

---

Answer: Referring to the figure, and focusing on positive examples (the same argument applies to negative examples with $y_i = -1$), we see the different positions for a positive example relative to the margin and hyperplane. The slack $\zeta_i$ ($Z_i$ in the figure) for example $i$ is the distance from the example to the positive (upper-right) margin.

![](https://drive.google.com/uc?id=1YtpoLHCCxWlL7BpUR13Oc0R0ZmZdcmlv)

For $\zeta_i < 0$, like example #1, it is well-beyond the margin and correctly classified as a positive example. Note that the SVM constrains all $\zeta_i \geq 0$, so this $\zeta_1$ would be set to zero in the minimization equation (7.38).

For $\zeta_i = 0$, like example #2, it is on the margin and correctly classified as a positive example. 

For $0 < \zeta_i < 1$, like example #3, it is inside the margin, but still correctly classified as a positive example.

For $\zeta_i = 1$, like example #4, it is right on the hyperplane. Typically points on the hyperplane are classified as positive, but it could go either way.

For $1 < \zeta_i < 2$, like example #5, it is within the margin of the hyperplane, but on the wrong side, so it will be incorrectly classified as a negative example.

For $\zeta_i = 2$, like example #6, it is on the margin, but on the wrong side of the hyperplane, so it will be incorrectly classified as a negative example.

And for $\zeta_i > 2$, like example #7, it is well beyond the margin on the wrong side of the hyperplane, so it will be incorrectly classified as a negative example.


---

#2.

(12 points) Suppose the two-layer neural network shown below processes the input (0, 1, 1, 0). If the actual output should be 0.2, show step-by-step how the vector of weights *v* will be updated using backpropagation and $\eta = 0.2$.

![](https://drive.google.com/uc?id=1mLkFgXA0drWp6nYL50n0BZv2Z13EA9CN)

---

Answer: The activation of the hidden units is calculated using the formula $a_i = w_i . x$, so the activation for hidden units 1, 2, and 3 (numbered top to bottom) is $tanh(1.0 \times 0 + 1.0 \times 1 + 1.0 \times 1 + 1.0 \times 0) = tanh(2.0) = 0.964$.

The output of the network is next calculated using the formula $v . h$, which equals $-0.4 \times 0.964 + 1.0 \times 0.964 + 0.2 \times 0.964 =  0.7712$. The error $e = 0.2 - 0.7712 = -0.5712$. The vector of weights *v* is updated using the formula $v = v - \eta g$ where $g = g - eh$. Here $g = 0.0 + 0.5712 \times 0.964 = 0.5506$ for the output layer and $v_1 = -0.4 - 0.2 \times 0.5506 = -0.5101$, $v_2 = 1.0 - 0.2 \times 0.5506 = 0.8899$, and $v_3 = 0.2 - 0.2 \times 0.5506 = 0.0899$.

To completely update the network we would next need to update the vector of weights $W$ using information that includes the error of the network, the previous $W$, the learning rate, and the output of the hidden node to which the edge points.

---

#3. 

(8 points) Under which of these conditions does an ensemble classifier perform best? There can be more than one right answer, explain all of your responses.

- Low prediction correlation between base classifiers.
- High prediction correlation between base classifiers.
- Base classifiers have low variance.
- Base classifiers have high bias.
- Base classifiers have high variance.

---

Answer: A lower correlation among base classifiers will increase the error-correcting capability of the ensemble, so this situation is preferred.

Weak learners are typically used in ensemble methods. They have low variance and do not typically overfit the training data. They also typically have low bias.

---

#4.

(80 points) The goal of this problem is for you to implement backpropagation from scratch. You can make use of python libraries for handling the data and computation, but implement the actual activation and weight change calculations yourself.

Test your neural network using the MNIST dataset. Information on loading and storing this handwritten-digit dataset can be found at https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html. Only consider digit classes '0' (which you can map onto value -1) and '1' (which you can map onto value 1). Train the network on a randomly-selected 2/3 of the data points and test on the remaining 1/3. You can report mean squared error or accuracy for the test data for a minimum of 10 epochs.


In [ ]:
from math import exp
from random import random
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import numpy as np


def initialize_network(n_inputs, n_hidden, n_outputs):
  network = list()  # initialize weights to random number in [0..1]
  hidden_layer = [{'weights':[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
  network.append(hidden_layer)
  output_layer = [{'weights':[random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
  network.append(output_layer)
  return network


def activate(weights, inputs):
  activation = weights[-1]   # bias
  for i in range(len(weights)-1):
    activation += weights[i] * inputs[i]
  return activation


#def transfer(activation): #sigmoid function
#  return 1.0 / (1.0 + exp(-activation))


def transfer(activation): # tanh function
  return np.tanh(activation)


def forward_propagate(network, X, y):
  inputs = X
  for layer in network:
    new_inputs = []
    for node in layer:
      activation = activate(node['weights'], X)
      node['output'] = transfer(activation)
      new_inputs.append(node['output']) # output of one node input to another
    inputs = new_inputs
  return inputs   # return output from last layer


#def transfer_derivative(output): # derivative of sigmoid function
#  return output * (1.0 - output)


def transfer_derivative(output): # derivative of tanh function
  return 1.0 - np.tanh(output)**2


def backward_propagate_error(network, expected):
  for i in reversed(range(len(network))): # from output back to input layers
    layer = network[i]
    errors = list()
    if i != len(network)-1:  # not the output layer
      for j in range(len(layer)):
        error = 0.0
        for node in network[i+1]:
          error += (node['weights'][j] * node['delta'])
        errors.append(error)
    else:   # output layer
      for j in range(len(layer)):
        node = layer[j]
        errors.append(expected[j] - node['output'])
    for j in range(len(layer)):
      node = layer[j]
      node['delta'] = errors[j] * transfer_derivative(node['output'])


def update_weights(network, x, y, eta):
  for i in range(len(network)):
    inputs = x
    if i != 0:
      inputs = [node['output'] for node in network[i-1]]
    for node in network[i]:
      for j in range(len(inputs)):
        node['weights'][j] += eta * node['delta'] * inputs[j]
      node['weights'][-1] += eta * node['delta']


def train_network(network, X, y, eta, num_epochs, num_outputs):
  for epoch in range(num_epochs):
    sum_error = 0
    for i in range(len(y)):
      outputs = forward_propagate(network, X[i], y[i])
      expected = [0 for i in range(num_outputs)]
      expected[y[i]] = 1
      sum_error += sum([(expected[i] - outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weights(network, X[i], y[i], eta)    
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, eta, sum_error))


def test_network(network, X, y, num_outputs):
  sum_error = 0
  for i in range(len(y)):
    outputs = forward_propagate(network, X[i], y[i])
    expected = [0 for i in range(num_outputs)]
    expected[y[i]] = 1
    sum_error += sum([(expected[i] - outputs[i])**2 for i in range(len(expected))])
  print('mse of test data is', sum_error / float(len(y)))


if __name__ == "__main__":
  # Load data from https://www.openml.org/d/554
  features, targets = fetch_openml('mnist_784', version=1, return_X_y=True)
  X = []
  y = []
  for i in range(len(targets)):
    if targets[i] == '1' or targets[i] == '0':
      X.append(features[i])
      if targets[i] == '0':
        y.append(-1)
      else:
        y.append(1)
  n_inputs = len(X[0])
  n_outputs = 2  # possible class values are 1 and -1
  network = initialize_network(n_inputs, 2, n_outputs)
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67, test_size=0.33)
  # train network for 20 epochs using learning rate of 0.5 
  train_network(network, X_train, y_train, 0.1, 10, n_outputs)
  for layer in network:
    print('layer \n', layer)
  test_network(network, X_test, y_test, n_outputs)

>epoch=0, lrate=0.100, error=7.697
>epoch=1, lrate=0.100, error=0.007
>epoch=2, lrate=0.100, error=0.002
>epoch=3, lrate=0.100, error=0.001
>epoch=4, lrate=0.100, error=0.001
>epoch=5, lrate=0.100, error=0.000
>epoch=6, lrate=0.100, error=0.000
>epoch=7, lrate=0.100, error=0.000
>epoch=8, lrate=0.100, error=0.000
>epoch=9, lrate=0.100, error=0.000
layer 
 [{'weights': [0.8413405274451046, 0.8842392084417927, 0.0036093158629025845, 0.01945574890330526, 0.7894074403097004, 0.12841866486546616, 0.19154272738940936, 0.9359190786623915, 0.8424409066840297, 0.531417481364686, 0.569306432364971, 0.9178088151756087, 0.7916815907665887, 0.5909954370083206, 0.7732657686626017, 0.2535426351413764, 0.19621811599998074, 0.2781936363669826, 0.5321457340751709, 0.5986871784547896, 0.9357316029837539, 0.5889611447929337, 0.19683236008811167, 0.3291682981132623, 0.3148724915421307, 0.8430706678548441, 0.14605936722305524, 0.10761180540487891, 0.5635018301475017, 0.7263938942366962, 0.6666608518452178, 